# Utilities

In [1]:
import nltk
from nltk.corpus import wordnet
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../')
from commons_init import SYNSET_DA_EVITARE, SUPPORTED_POS

## Calcolo della vicinanza LCH
La similarità tra due synset, misurata tramite l'indice Leacock-Chodorow (LCH), influisce positivamente sulla valutazione della *basicness*. Maggiore è la similarità tra i synset, maggiore sarà il contributo alla basicness.

In WordNet (WN), la vicinanza si basa sulla gerarchia dei synset. L'indice LCH misura la somiglianza tra due concetti in base alla lunghezza del percorso più breve che li separa nella gerarchia di WN.

- LCH quantifica la vicinanza tra due concetti: valori più alti indicano una maggiore similarità, mentre valori più bassi indicano una minore similarità.
- Per ogni synset, si calcola la distanza rispetto al synset di base per quella parte del discorso (POS) in WN.
- Un valore LCH vicino a 1 indica una forte similarità tra il synset esaminato e il concetto di base, segnalando che il synset è considerato più *basic*.
- Synset con un elevato punteggio di similarità LCH sono giudicati più *basic*, e pertanto ricevono uno score di basicness più alto.

In [3]:
base_concept = {
    'n': wordnet.synset('entity.n.01'),
    'v': wordnet.synset('act.v.01'),
    'a': wordnet.synset('good.a.01'),
    'r': wordnet.synset('well.r.01')
}

In [4]:
def compute_similarity_LCH(synset, postag, base_concept):
    concept_base = base_concept[postag]
    
    lch_similarity = concept_base.lch_similarity(synset)
    
    min_value = 0.0
    max_value = concept_base.lch_similarity(concept_base) # massimo valore possibile per LCH similarity (3.6375861597263857)
    normalized_lch_similarity = (lch_similarity - min_value) / (max_value - min_value)
    
    return lch_similarity, normalized_lch_similarity

## Calcolo vicinanza LCH dei synset

In [5]:
def compute_synsets_similarity():
    synset_similarity_dict = {}
    
    for pos in SUPPORTED_POS:
        for synset in wordnet.all_synsets(pos):
            
            try:
                if synset in SYNSET_DA_EVITARE or synset.pos() == 's':
                    continue  
                
                lch_similarity, normalized_lch_similarity = compute_similarity_LCH(synset, pos, base_concept)
                
                synset_similarity_dict[synset] = lch_similarity, normalized_lch_similarity
            
            except StopIteration:
                break
            
    return synset_similarity_dict

In [6]:
synset_similarity_dict = compute_synsets_similarity()

In [7]:
data_df = {
    'Synset': [str(key) for key in synset_similarity_dict.keys()],
    'LCH Similarity': [item[0] for item in synset_similarity_dict.values()],
    'Normalized LCH Similarity': [item[1] for item in synset_similarity_dict.values()]
}
df_similarity = pd.DataFrame(data_df)

## Similarità di un synset specifico

In [8]:
synset = wordnet.synset('animal.n.01')
similarity_dict = synset_similarity_dict.get(synset, 'Synset non classificato')
sim = similarity_dict[0]
sim_norm = similarity_dict[1]
print(sim)
print(sim_norm)

1.6916760106710724
0.4650545544186692


## Salvataggio su file

In [9]:
path_output = "../features/df/"
df_file_output = path_output + 'df_similarity.csv'
df_similarity.to_csv(df_file_output, index=False)

# Risultati ottenuti

In [10]:
print(df_similarity[:10])

                           Synset  LCH Similarity  Normalized LCH Similarity
0           Synset('entity.n.01')        3.637586                   1.000000
1  Synset('physical_entity.n.01')        2.944439                   0.809449
2      Synset('abstraction.n.06')        2.944439                   0.809449
3            Synset('thing.n.12')        2.538974                   0.697983
4           Synset('object.n.01')        2.538974                   0.697983
5            Synset('whole.n.02')        2.251292                   0.618897
6         Synset('congener.n.03')        2.028148                   0.557553
7     Synset('living_thing.n.01')        2.028148                   0.557553
8         Synset('organism.n.01')        1.845827                   0.507432
9          Synset('benthos.n.02')        1.691676                   0.465055
